In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import re
import urllib.request
from collections import Counter 
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt
from konlpy.tag import Mecab

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, Dropout, BatchNormalization, Conv1D, GlobalAveragePooling1D, Bidirectional, LSTM, Flatten, MaxPooling1D, Reshape
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
import pandas as pd
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

df = pd.read_csv('../data/title_for_ml_1.csv')

pandas version:  1.3.4


In [3]:
print(df.shape)

(499999, 4)


In [5]:
import re

def textOptimize(text) :
  text = text.lower()
  text = re.sub("(a|c)([^0-9a-zA-Z가-힣])s", "\\1s", text)
  text = re.sub("(\d{3})[^0-9a-zA-Z가-힣]*(\d{3,4})[^0-9a-zA-Z가-힣]*(\d{3,4})", " ", text)
  text = re.sub("(\d{2})( )*:( )*(\d{2})", "\\1:\\4", text)
  text = re.sub("[a-z0-9_+.-]+@([a-z0-9-]+\.)+[a-z0-9]{2,4}", " ", text)
  text = re.sub("&#[0-9]*;", "", text)
  text = re.sub(":(?!\d{2}\D)", "", text)
  text = re.sub("&(?!b )", " ", text) 
  text = re.sub("[^0-9a-zA-Z가-힣:& ]", " ", text)
  text = re.sub("(\d{1,3})[., ](\d{3})", "\\1\\2", text)
  text = re.sub("(\d{1,3})[., ](\d{3})", "\\1\\2", text)
  text = text.replace("\t", " ")
  text = re.sub(" +"," ", text)
  
  text = text.replace("알바천국", "")
  text = text.replace("알 바 천 국", "")
  text = re.sub("^ ","", text)

  return str(text)

In [8]:
import re

test = []

for i in range(len(df)):
  text = str(df.TITLE_analyze[i])
  text = re.sub("[^0-9a-zA-Z가-힣:& ]", " ", text)
  text = re.sub(" +"," ", text)
  text = re.sub("^ ","", text)
  test.append(text)

df['TEST'] = test

In [10]:
df.head()

,TITLE,JK_NAME,TITLE_analyze,JK_NAME_analyze,TEST
0,월200만원 a 신한카드 설계사및 신규상품 홍보팀원모집 투잡가능,고객상담·인바운드,월 200 만 원 a 신한 카드 설계 사 및 신규 상품 홍보 팀 원 모집 투잡 가능,고객 상담 인 바운드,월 200 만 원 a 신한 카드 설계 사 및 신규 상품 홍보 팀 원 모집 투잡 가능
1,월200만원 a 신한카드 설계사및 신규상품 홍보팀원모집 투잡가능,금융·보험영업,월 200 만 원 a 신한 카드 설계 사 및 신규 상품 홍보 팀 원 모집 투잡 가능,금융 보험 영업,월 200 만 원 a 신한 카드 설계 사 및 신규 상품 홍보 팀 원 모집 투잡 가능
2,월220 준오헤어 강남1호점 데스크 매장관리 직원,반려동물케어,월 220 준오 헤어 강남 1 호 점 데스크 매장 관리 직원,반려 동물 케어,월 220 준오 헤어 강남 1 호 점 데스크 매장 관리 직원
3,월230만원 일8시간 주6일근무 주방직원 구함,패스트푸드,월 230 만 원 일 8시간 주 6일 근무 주방 직원 구함,패스트 푸드,월 230 만 원 일 8시간 주 6일 근무 주방 직원 구함
4,월235 경력무관 준오헤어 서울대입구역점 패션스트릿매장 카운터 관리자 매장관리 고속승진,반려동물케어,월 235 경력 무관 준오 헤어 서울대 입구 역 점 패션 스트릿 매장 카운터 관리자...,반려 동물 케어,월 235 경력 무관 준오 헤어 서울대 입구 역 점 패션 스트릿 매장 카운터 관리자...


In [ ]:
import re

test = []

for i in range(len(df)):
  text = str(df.TITLE_analyze[i])
  text = re.sub("[^0-9a-zA-Z가-힣:& ]", " ", salary)
  text = re.sub(" +"," ", text)
  text = re.sub("^ ","", text)
  test.append(text)

df['TEST'] = test

In [12]:
df.head()

,TITLE,JK_NAME,TITLE_analyze,JK_NAME_analyze,TEST
0,월200만원 a 신한카드 설계사및 신규상품 홍보팀원모집 투잡가능,고객상담·인바운드,월 200 만 원 a 신한 카드 설계 사 및 신규 상품 홍보 팀 원 모집 투잡 가능,고객 상담 인 바운드,월 200 만 원 a 신한 카드 설계 사 및 신규 상품 홍보 팀 원 모집 투잡 가능
1,월200만원 a 신한카드 설계사및 신규상품 홍보팀원모집 투잡가능,금융·보험영업,월 200 만 원 a 신한 카드 설계 사 및 신규 상품 홍보 팀 원 모집 투잡 가능,금융 보험 영업,월 200 만 원 a 신한 카드 설계 사 및 신규 상품 홍보 팀 원 모집 투잡 가능
2,월220 준오헤어 강남1호점 데스크 매장관리 직원,반려동물케어,월 220 준오 헤어 강남 1 호 점 데스크 매장 관리 직원,반려 동물 케어,월 220 준오 헤어 강남 1 호 점 데스크 매장 관리 직원
3,월230만원 일8시간 주6일근무 주방직원 구함,패스트푸드,월 230 만 원 일 8시간 주 6일 근무 주방 직원 구함,패스트 푸드,월 230 만 원 일 8시간 주 6일 근무 주방 직원 구함
4,월235 경력무관 준오헤어 서울대입구역점 패션스트릿매장 카운터 관리자 매장관리 고속승진,반려동물케어,월 235 경력 무관 준오 헤어 서울대 입구 역 점 패션 스트릿 매장 카운터 관리자...,반려 동물 케어,월 235 경력 무관 준오 헤어 서울대 입구 역 점 패션 스트릿 매장 카운터 관리자...


In [17]:
df.isnull().sum()

TITLE              0
JK_NAME            0
TITLE_analyze      0
JK_NAME_analyze    0
TEST               0
dtype: int64

In [36]:
df2 = df[['JK_NAME', 'TEST']].sample(frac=1).reset_index(drop=True)
df2 = df[df.TEST.str.len() > 5]
df.reset_index(drop=True, inplace=True)

df_idx = df.groupby(['JK_NAME'])['JK_NAME'].count().reset_index(name='count')
df_idx = df_idx[df_idx['count'] > 10].reset_index(drop=True)
df_idx['idx'] = pd.Series(np.arange(0,len(df_idx)))

df = df.merge(df_idx[['JK_NAME', 'idx']], on='JK_NAME')
df = df.sample(frac=1).reset_index(drop=True)
df

,TITLE,JK_NAME,TITLE_analyze,JK_NAME_analyze,TEST,idx
0,강추 홀서빙 직원 및 홀 알바 구함,일반음식점,강추 홀 서빙 직원 및 홀 알바 구함,일반 음식 점,강추 홀 서빙 직원 및 홀 알바 구함,99
1,gs25 인천문학점 일 월 화 야간 근무자 모집,매장관리·판매,gs 25 인천 문학 점 일 월 화 야간 근무 자 모집,매장 관리 판매,gs 25 인천 문학 점 일 월 화 야간 근무 자 모집,38
2,신규오픈 신세계백화점 대전점 니치매장 백화점 신입 경력직 명품매장사원 매둘막,뷰티·헬스스토어,신규 오픈 신세계 백화점 대전 점 니치 매장 백화점 신입 경력 직 명품 매장 사원 ...,뷰티 헬스 스토어,신규 오픈 신세계 백화점 대전 점 니치 매장 백화점 신입 경력 직 명품 매장 사원 ...,61
3,gs25유니온점 알바직원 구함 유니온 빌리지 상가내,캐셔·카운터,gs 25 유니온 점 알바 직원 구함 유니온 빌리지 상가 내,캐셔 카운터,gs 25 유니온 점 알바 직원 구함 유니온 빌리지 상가 내,119
4,gs25 반여재반로점 스토어매니저 모집합니다 주말 야간,매장관리·판매,gs 25 반여 재반 로 점 스토어 매니저 모집 하 ᄇ니다 주말 야간,매장 관리 판매,gs 25 반여 재반 로 점 스토어 매니저 모집 하 니다 주말 야간,38
...,...,...,...,...,...,...
499976,홀써빙주방 평일오전 오후주방직원및 주말알바구해요,일반음식점,홀 써빙 주방 평일 오전 오후 주방 직원 및 주말 알바 이 구 하 아요,일반 음식 점,홀 써빙 주방 평일 오전 오후 주방 직원 및 주말 알바 이 구 하 아요,99
499977,스튜디오 사진촬영 촬영보조 직원 채용,방송스텝·촬영보조,스튜디오 사진 촬영 촬영 보조 직원 채용,방송 스텝 촬영 보조,스튜디오 사진 촬영 촬영 보조 직원 채용,48
499978,nc백화점 불광점 킴스 정육 양념육 코너 직원급구,대형마트,nc 백화점 불광 점 킴스 정육 양념 육 코너 직원 급구,대형 마트,nc 백화점 불광 점 킴스 정육 양념 육 코너 직원 급구,28
499979,gs25 대구내당광장점 스토어매니저 모집,편의점,gs 25 대구 내당 광장 점 스토어 매니저 모집,편의점,gs 25 대구 내당 광장 점 스토어 매니저 모집,130


In [37]:
df_idx.sort_values(by=['count'], ascending=False)

,JK_NAME,count,idx
130,편의점,84751,130
99,일반음식점,37080,99
38,매장관리·판매,22939,38
61,뷰티·헬스스토어,20890,61
142,화물·중장비·특수차,19115,142
26,대리운전·일반운전,16912,26
119,캐셔·카운터,16688,119
111,주방보조·설거지,14150,111
49,배달,12815,49
123,택배·퀵서비스,12515,123


In [34]:
df['JK_NAME_analyze'].value_counts()

편의점                 84751
일반 음식 점             37080
매장 관리 판매            22939
뷰티 헬스 스토어           20890
화물 중 장비 특수 차        19115
대리 운전 일반 운전         16912
캐셔 카운터              16688
주방 보조 설거지           14150
배달                  12815
택배 퀵 서비스            12515
제조 가공               11965
사무 보조               11240
포장 품질 검사            11107
운반 이사                9947
대형 마트                8997
고객 상담 인 바운드          8191
생산 건설 노무 기타          7968
커피 전문점               7623
서비스 기타               7141
의류 잡화 매장             6666
입 출고 창고 관리           6655
pc 방                 5344
공사 건설 현장             5000
입시 보습 학원             4964
패스트 푸드               4947
휴대폰 전자 기기 매장         4889
방문 학습 지 교사           4865
텔레 마케팅 아웃 바운드        4687
서빙                   4064
복합 쇼핑 몰 아울렛          3886
유통 판매 기타             3577
백화점 면세 점             3406
바 bar                3404
일반 영업 판매             3351
전기 전자 가스             3294
반려 동물 케어             3217
호프 일반 주점             3120
농 수산 청과 축산           3063
급식 푸드 시스템   

In [32]:
df1=df['JK_NAME_analyze'].value_counts()

In [33]:
df1.describe()

count      148.000000
mean      3378.371622
std       8491.664980
min          8.000000
25%        158.000000
50%        597.000000
75%       3077.250000
max      84751.000000
Name: JK_NAME_analyze, dtype: float64

### 데이터 전처리 : 카테고리별 편차가 너무 커서 Z-점수 정규화 시킬것임  
* Z = (X-평균) / (표준편차)

### 데이터 전처리 : 토큰화

In [38]:
x_data = df['TEST']
y_data = df['idx']

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_data) ## 토큰들을 숫자로 변환

sequences = tokenizer.texts_to_sequences(x_data) ## 시퀀스 형태로 변환

x_data = sequences

max_len=50 ## 모델 학습을 위해 모든 input data 길이 동일하게 설정
x_train = pad_sequences(x_data, maxlen=max_len, padding='post')
y_train = to_categorical(y_data)


In [ ]:
df.head()

### 예측 모델 구현

 - tensorflow 사용
 - 모델 구조: CNN-LSTM
 - loss: categorical_crossentropy
 - optimizer: adam

In [40]:
model = Sequential()

model.add(Embedding(len(tokenizer.word_index)+1, 300, input_length=max_len))
model.add(BatchNormalization())
model.add(Dropout(.2))
model.add(Conv1D(300, 3, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(200)))
model.add(BatchNormalization())
model.add(Dropout(.2))
model.add(Dense(len(y_data.unique()), activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=6)
mc = ModelCheckpoint('cnnlstm_for_comnm_title_220221.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
model.fit(x_train, y_train, batch_size=512, epochs=100, callbacks=[es, mc], validation_split=0.2)

Epoch 1/100
782/782 [==============================] - ETA: 0s - loss: 1.6519 - acc: 0.5235
Epoch 1: val_acc improved from -inf to 0.54921, saving model to cnnlstm_for_comnm_title_220221.h5
782/782 [==============================] - 1664s 2s/step - loss: 1.6519 - acc: 0.5235 - val_loss: 1.4053 - val_acc: 0.5492
Epoch 2/100
782/782 [==============================] - ETA: 0s - loss: 1.2342 - acc: 0.5931
Epoch 2: val_acc improved from 0.54921 to 0.58196, saving model to cnnlstm_for_comnm_title_220221.h5
782/782 [==============================] - 1519s 2s/step - loss: 1.2342 - acc: 0.5931 - val_loss: 1.2608 - val_acc: 0.5820
Epoch 3/100
326/782 [===========>..................] - ETA: 13:39 - loss: 1.0813 - acc: 0.6282

### 성능좋은 모델 찾기

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action = 'ignore')

from matplotlib import font_manager, rc 
plt.rcParams['axes.unicode_minus'] = False

f_path = "c:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=f_path).get_name()
rc('font', family=font_name)

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, classification_report, roc_curve, plot_roc_curve, auc, precision_recall_curve, plot_precision_recall_curve, average_precision_score
from sklearn.model_selection import cross_val_score

In [ ]:
models = []
models.append(['Logistic Regreesion', LogisticRegression(random_state=0)])
models.append(['SVM', SVC(random_state=0)])
models.append(['KNeighbors', KNeighborsClassifier()])
models.append(['GaussianNB', GaussianNB()])
models.append(['BernoulliNB', BernoulliNB()])
models.append(['Decision Tree', DecisionTreeClassifier(random_state=0)])
models.append(['Random Forest', RandomForestClassifier(random_state=0)])
models.append(['Extra Tree', ExtraTreesClassifier(random_state=0)])
models.append(['AdaBoost', AdaBoostClassifier(random_state=0)])
models.append(['Gradiesnt Boost', GradientBoostingClassifier(random_state=0)])
models.append(['Light GBM', LGBMClassifier(random_state=0)])
models.append(['XGBoost', XGBClassifier(eval_metric= 'error')])

lst_1= []

for m in range(len(models)):
    lst_2= []
    model = models[m][1]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)  #혼동행렬
    accuracies = cross_val_score(estimator = model, X = x_train, y = y_train, cv = 10)   #K-Fold Validation
    roc = roc_auc_score(y_test, y_pred)  #ROC AUC Score
    precision = precision_score(y_test, y_pred)  #Precision Score
    recall = recall_score(y_test, y_pred)  #Recall Score
    f1 = f1_score(y_test, y_pred)  #F1 Score
    print(models[m][0],':')
    print(cm)
    print('Accuracy Score: ',accuracy_score(y_test, y_pred))
    print('')
    print("K-Fold Validation Mean Accuracy: {:.4f} %".format(accuracies.mean()*100))
    print('')
    print("Standard Deviation: {:.4f} %".format(accuracies.std()*100))
    print('')
    print('ROC AUC Score: {:.4f}'.format(roc))
    print('')
    print('Precision: {:.4f}'.format(precision))
    print('')
    print('Recall: {:.4f}'.format(recall))
    print('')
    print('F1: {:.4f}'.format(f1))
    print('-----------------------------------')
    print('')